In [1]:
import copy

import pandas as pd

C:\Users\flavi\AppData\Local\Temp\ipykernel_18532\3074791949.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_og = pd.read_pickle("MulTweEmo/datasetPkl/merged_df_with_gold_freq1.pkl")

In [3]:
def anonymize_and_fix_amps(tweet):
    toks = str(tweet).split()
    for idx,t in enumerate(toks):
        if t[0] == '@':
            toks[idx] = '@USER'
        if t == '&amp;':
            toks[idx] = '&'
    return ' '.join(toks)

df_og['tweet'] = df_og['tweet'].apply(anonymize_and_fix_amps)
df_gold = df_og[df_og.T_Surprise.isnull() == False]
df_gold = df_gold[df_gold.M_gold_multi_label.str.len() != 0]

In [4]:
def remove_neutral_smt(labels):
    new_l=[]
    for e in labels:
        if e != "Something else" and e != "Neutral":
            new_l.append(e)
    return new_l

In [5]:
df_gold['M_gold_multi_label'] = df_gold['M_gold_multi_label'].apply(remove_neutral_smt)
df_gold['T_gold_multi_label'] = df_gold['T_gold_multi_label'].apply(remove_neutral_smt)

In [6]:
len(df_gold)

875

In [7]:
df = df_gold

In [8]:
len(df)

875

In [9]:
def replace_string(row):
    new_row = []
    for item in row:
        new_row.append(item.replace('../',''))
    return new_row

df['path_photos'] = df['path_photos'].apply(replace_string)

In [10]:
columns = ['tweet','path_photos', 'M_gold_multi_label', 'T_gold_multi_label']
df = df.loc[:, columns]

In [11]:
df = df.convert_dtypes()

In [12]:
df = df.explode(['path_photos']).reset_index(drop=True)

In [13]:
df = df[:500]

In [14]:
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import tensorflow as tf

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
device = "cuda:0" if torch.cuda.is_available() else "cpu" 

def extract_and_concatenate(curr_txt, curr_img):
    curr_img = Image.open(curr_img)
    inputs = processor(text=curr_txt, 
                       images=curr_img, 
                       return_tensors='pt', 
                       padding='max_length', 
                       truncation=True)
    inputs = inputs
    outputs = model(**inputs)
    image_features = outputs.image_embeds
    text_features = outputs.text_embeds
    
    concat = torch.cat((image_features, text_features), dim=1)
    #concat = tf.concat(values=[image_features, text_features], axis=1)
    return concat

images = df.path_photos.values
texts = df.tweet.values

X = []

for idx, p in enumerate(images):
    curr_img = images[idx]
    curr_txt = texts[idx]
    print(idx)
    conc = extract_and_concatenate(curr_txt, curr_img)
    X.append(conc)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer

X = np.squeeze(np.asarray([i.detach().numpy() for i in X]))

y = df['T_gold_multi_label']

mlb = MultiLabelBinarizer()
yt = mlb.fit_transform(y)

In [16]:
len(X), len(yt)

(500, 500)

In [17]:
from keras import backend as K
from keras.layers import Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, schedules, SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM
import tensorflow as tf 

tf.config.run_functions_eagerly(True)

#keras custom f1_score metric
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric."""

        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric."""

        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
        
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, yt, test_size=0.15,
                                                   shuffle=True, random_state=42)

model_1 = Sequential(name="ANN-1")
model_1.add(Input(shape=(1024,), name='input'))  # is 512 when using only visual or textual embeddings
                                 # & 1024 when using multimodal embeddings
model_1.add(Dense(400, activation='relu', name="feedforward_1"))
model_1.add(Dropout(0.4, name='dropout_0.4'))

model_1.add(Dense(200, activation='relu', name="feedforward_2"))
model_1.add(Dropout(0.2, name="dropout_0.2"))
model_1.add(Dense(8, activation='sigmoid', name="output"))
model_1.output_shape

#opt = tf.keras.optimizers.SGD(learning_rate=0.002, nesterov=True)
model_1.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=[f1, "accuracy"],)

In [18]:
es = EarlyStopping(monitor='val_loss', patience=20, verbose=1)
BATCH = int(len(X_train)/10)

print('Train...')
model_1.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=300,
          validation_split=0.15,
          shuffle=True,
          callbacks=[es],
          )

Train...


C:\Users\flavi\thesis\venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/300
9/9 [==============================] - 1s 90ms/step - loss: 0.6639 - f1: 0.1088 - accuracy: 0.1967 - val_loss: 0.6344 - val_f1: 0.0000e+00 - val_accuracy: 0.1562
Epoch 2/300
9/9 [==============================] - 1s 74ms/step - loss: 0.5973 - f1: 0.1328 - accuracy: 0.2770 - val_loss: 0.6062 - val_f1: 0.2660 - val_accuracy: 0.1562
Epoch 3/300
9/9 [==============================] - 1s 77ms/step - loss: 0.5635 - f1: 0.2990 - accuracy: 0.2770 - val_loss: 0.5899 - val_f1: 0.3005 - val_accuracy: 0.1562
Epoch 4/300
9/9 [==============================] - 1s 76ms/step - loss: 0.5297 - f1: 0.3460 - accuracy: 0.2770 - val_loss: 0.5645 - val_f1: 0.3091 - val_accuracy: 0.1562
Epoch 5/300
9/9 [==============================] - 1s 74ms/step - loss: 0.4881 - f1: 0.3925 - accuracy: 0.3047 - val_loss: 0.5381 - val_f1: 0.3896 - val_accuracy: 0.2031
Epoch 6/300
9/9 [==============================] - 1s 78ms/step - loss: 0.4420 - f1: 0.5420 - accuracy: 0.4100 - val_loss: 0.5088 - val_f1: 0.4526

In [19]:
predict= model_1.evaluate(x=X_test, y=y_test)

print("Keras F1")
print("ANN-1:")
print("Loss:"+str(predict[0]))
print("F1:"+str(predict[1]))
print("Accuracy:"+str(predict[2]))

from sklearn.metrics import f1_score

y_pred = model_1.predict([X_test], verbose=3)

result_1 = f1_score(y_true=y_test.round(), y_pred=y_pred.round(), labels=None, average="weighted")

from sklearn.metrics import classification_report
labels=mlb.classes_
print(classification_report(y_true=y_test.round(), y_pred=y_pred.round(), target_names=labels))

3/3 [==============================] - 0s 23ms/step - loss: 0.6431 - f1: 0.6679 - accuracy: 0.3733


C:\Users\flavi\thesis\venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Keras F1
ANN-1:
Loss:0.6430535316467285
F1:0.6678892970085144
Accuracy:0.3733333349227905
              precision    recall  f1-score   support

       Anger       0.64      0.56      0.60        16
Anticipation       0.64      0.53      0.58        30
     Disgust       0.69      0.45      0.55        20
        Fear       0.75      0.60      0.67        10
         Joy       0.88      0.91      0.89        46
     Sadness       0.57      0.36      0.44        22
    Surprise       0.65      0.55      0.59        20
       Trust       0.83      0.53      0.65        19

   micro avg       0.74      0.61      0.66       183
   macro avg       0.71      0.56      0.62       183
weighted avg       0.72      0.61      0.65       183
 samples avg       0.71      0.63      0.64       183


C:\Users\flavi\thesis\venv\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
